In [1]:
from selenium import webdriver
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

import re
import datetime
import locale

In [2]:
def getData(url):
    service = Service('C:/Users/alexa/Downloads/chromedriver-win64/chromedriver.exe')
    driver = webdriver.Chrome(service=service)

    driver.get(url)

    main_element = driver.find_element(By.ID, 'mainContent')

    scripts = main_element.find_elements(By.TAG_NAME, 'script')

    json_data = None

    for script in scripts:
        if 'application/ld+json' in script.get_attribute('type'):
            json_data = script.get_attribute('innerHTML')
            break

    driver.quit()

    if json_data:
        data = json.loads(json_data)
    
    else:
        print("pas de json")
        
    return data

In [3]:
def getInfo(data):
    menu = []
    
    for menus in data['hasMenu']:
        for section in menus['hasMenuSection']:
            for item in section['hasMenuItem']:
                menu.append([item['name'],item['description']])
                
    rating = data['aggregateRating']['ratingValue'] 
    priceRange = data['priceRange']
    localisation = data['address']['addressLocality']
    nom = data['name'] 
    coords = [data['geo']['latitude'],data['geo']['longitude']]
    typeCuisine = data['servesCuisine'].split(", ")
    
    result_string = ' '.join(word for sublist in menu if sublist is not None for sentence in sublist if sentence is not None for word in sentence.split() if word)
    fruit_score, legume_score, cereale_score = getScores(result_string)
    
    
    resto = pd.DataFrame({"nom" : nom, "typeCuisine" : [typeCuisine], "localisation":localisation, "priceRange":priceRange, "rating":rating, "menu":[menu], "coords":[coords],"fruit_score":fruit_score, "legume_score":legume_score, "cereale_score":cereale_score})
    return resto

In [4]:
def getScores(result_string):
    locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')
    current_date = datetime.datetime.now()
    current_month_name = current_date.strftime("%B")
    
    current_legumes = noms_par_mois[current_month_name]["Legumes"]
    current_fruits = noms_par_mois[current_month_name]["Fruits"]
    current_cereales = noms_par_mois[current_month_name]["Céréales et légumineuses"]

    total_fruits_count = 0
    total_legumes_count = 0
    total_cereales_count = 0

    # Iterate through the translated lists and count occurrences
    for word in unique_fruits_list:
        total_fruits_count += result_string.lower().count(word.lower())

    for word in unique_legumes_list:
        total_legumes_count += result_string.lower().count(word.lower())

    for word in unique_cereales_list:
        total_cereales_count += result_string.lower().count(word.lower())

    
    total_current_fruits_count = 0
    total_current_legumes_count = 0
    total_current_cereales_count = 0

    # Iterate through the translated lists and count occurrences
    for word in current_fruits:
        total_current_fruits_count += result_string.lower().count(word.lower())

    for word in current_legumes:
        total_current_legumes_count += result_string.lower().count(word.lower())

    for word in current_cereales:
        total_current_cereales_count += result_string.lower().count(word.lower())

    
    fruit_score = total_current_fruits_count/total_fruits_count
    legume_score = total_current_legumes_count/total_legumes_count
    cereale_score = total_current_cereales_count/total_cereales_count

    return fruit_score, legume_score, cereale_score

Récuperation fruits/legumes

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
service = Service('C:/Users/alexa/Downloads/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service)
url = 'https://www.greenpeace.fr/guetteur/calendrier/' #mon site à scrapper

driver.get(url) # Ouvrir la page web avec Selenium

# Attendre que le contenu dynamique soit chargé
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "month")))

# stocker le code source de la page dans la variable htlm
html = driver.page_source

# Fermer le navigateur
driver.quit()

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
mois_elements = soup.find_all('li', class_='month')

In [9]:
noms_par_mois = {}
for mois in mois_elements:
    nom_du_mois = mois.find('a').get('id') # Récupérer les noms des mois
#     print(nom_du_mois)
    # Le premier 'ul' pour les légumes
    ul_legumes = mois.find_next('ul', attrs={'data-v-a603cee8': True})
    legumes = [li.get_text(strip=True) for li in ul_legumes.find_all('li', attrs={'data-v-a603cee8': True})]
#     print(legumes)
    #  Après celui des légumes, c'est le ul des fruits
    ul_fruits = ul_legumes.find_next('ul', attrs={'data-v-a603cee8': True})
    fruits = []
    if ul_fruits:
        fruits = [li.get_text(strip=True) for li in ul_fruits.find_all('li', attrs={'data-v-a603cee8': True})]
#     print(fruits)    
    # Et le dernier c'est celui des Céréales & légumineuses
    ul_cere = ul_fruits.find_next('ul', attrs={'data-v-a603cee8': True})
    cereales = []
    if ul_cere:
        cereales = [li.get_text(strip=True) for li in ul_cere.find_all('li', attrs={'data-v-a603cee8': True})]
#     print(cereales)
    # j'ajoute dans le dico avec le nom du mois comme clé
    noms_par_mois[nom_du_mois] = {'Legumes': legumes, 'Fruits': fruits, 'Céréales et légumineuses': cereales}

In [10]:
noms_par_mois

{'janvier': {'Legumes': ['Ail',
   'Betterave',
   'Carotte',
   'Céleri-branche',
   'Céleri-rave',
   'Chou',
   'Chou blanc',
   'Chou de Bruxelles',
   'Chou frisé',
   'Chou rouge',
   'Courge',
   'Crosne',
   'Endive',
   'Épinard',
   'Frisée',
   'Mâche',
   'Navet',
   'Oignon',
   'Panais',
   'Poireau',
   'Pomme de terre de conservation',
   'Salsifi',
   'Topinambour'],
  'Fruits': ['Amande sèche',
   'Citron',
   'Clémentine',
   'Kaki',
   'Kiwi',
   'Mandarine',
   'Orange',
   'Pamplemousse',
   'Physalis',
   'Poire',
   'Pomme'],
  'Céréales et légumineuses': ['Lentille']},
 'fevrier': {'Legumes': ['Ail',
   'Betterave',
   'Carotte',
   'Céleri-rave',
   'Chou blanc',
   'Chou de Bruxelles',
   'Chou frisé',
   'Chou rouge',
   'Crosne',
   'Endive',
   'Épinard',
   'Frisée',
   'Mâche',
   'Navet',
   'Oignon',
   'Panais',
   'Poireau',
   'Pomme de terre de conservation',
   'Radis',
   'Salsifi',
   'Topinambour'],
  'Fruits': ['Amande sèche',
   'Citron',
   

In [11]:
unique_fruits = set()
unique_legumes = set()
unique_cereales = set()

for mois_info in noms_par_mois.values():
    unique_fruits.update(mois_info.get('Fruits', []))
    unique_legumes.update(mois_info.get('Legumes', []))
    unique_cereales.update(mois_info.get('Céréales et légumineuses', []))

# Convert the sets to lists
unique_fruits_list = list(unique_fruits)
unique_legumes_list = list(unique_legumes)
unique_cereales_list = list(unique_cereales)

In [12]:
unique_fruits_list

['Mûre',
 'Framboise',
 'Brugnon',
 'Noisette',
 'Nectarine',
 'Physalis',
 'Myrtille',
 'Cerise',
 'Citron',
 'Cassis',
 'Pastèque',
 'Poire',
 'Noix',
 'Kaki',
 'Melon',
 'Mandarine',
 'Pêche',
 'Pomme',
 'Groseille',
 'Abricot',
 'Prune',
 'Clémentine',
 'Amande fraiche',
 'Châtaigne',
 'Kiwi',
 'Baie de goji',
 'Figue',
 'Pamplemousse',
 'Amande sèche',
 'Rhubarbe',
 'Coing',
 'Tomate',
 'Raisin',
 'Orange',
 'Mirabelle',
 'Pruneau',
 'Fraise']

In [13]:
unique_legumes_list

['Fenouil',
 'Petit pois',
 'Salsifi',
 'Ail',
 'Endive',
 'Chou blanc',
 'Brocoli',
 'Aubergine',
 'Céleri-branche',
 'Chou rouge',
 'Panais',
 'Poireau',
 'Chou romanesco',
 'Laitue',
 'Épinard',
 'Courge',
 'Frisée',
 'Crosne',
 'Chou',
 'Citrouille',
 'Betterave',
 'Poivron',
 'Echalote',
 'Chou frisé',
 'Céleri-rave',
 'Pomme de terre primeur',
 'Rutabaga',
 'Carotte',
 'Oignon',
 'Cardon',
 'Asperge',
 'Blette',
 'Courgette',
 'Mâche',
 'Artichaut',
 'Haricot vert',
 'Chou de Bruxelles',
 'Potiron',
 'Topinambour',
 'Pomme de terre de conservation',
 'Patate douce',
 'Navet',
 'Concombre',
 'Chou-fleur',
 'Radis']

In [14]:
unique_cereales_list

['Fenouil',
 'Petit pois',
 'Ail',
 'Riz',
 "Blé dur d'hiver et de printemps",
 'Brocoli',
 'Aubergine',
 'Haricot blanc',
 'Chou romanesco',
 'Orge de printemps',
 'Tournesol',
 'Laitue',
 'Épinard',
 'Maïs',
 "Avoine d'hiver",
 "Blé tendre d'hiver et de printemps",
 "Orge d'hiver",
 'Pois',
 'Avoine de printemps',
 'Betterave',
 'Poivron',
 'Pomme de terre primeur',
 'Féveroles et fèves',
 'Carotte',
 'Oignon',
 'Asperge',
 'Blette',
 'Courgette',
 'Quinoa',
 'Sarrasin',
 'Lentille',
 'Artichaut',
 'Haricot vert',
 'Soja',
 'Seigle',
 'Navet',
 'Concombre',
 'Chou-fleur',
 'Radis']

Recupération infos restaurant

In [15]:
url ='https://www.opentable.com/r/market-by-jean-georges-paris?corrid=216daf70-c02b-4cad-8d0c-2af83e2b87ce&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2024-01-09T19%3A00%3A00'
data = getData(url)

In [16]:
restaurants = pd.DataFrame(columns = ["nom", "typeCuisine", "localisation", "priceRange", "rating", "menu", "coords", "fruit_score", "legume_score","cereale_score"])

In [17]:
 #resto = pd.DataFrame({"nom" : nom, "typeCuisine" : [typeCuisine], "localisation":localisation, "priceRange":priceRange, "rating":rating, "menu":[menu], "coords":[coords]})

In [18]:
resto = getInfo(data)

In [19]:
restaurants = pd.concat([restaurants, resto])

In [20]:
restaurants

,nom,typeCuisine,localisation,priceRange,rating,menu,coords,fruit_score,legume_score,cereale_score
0,Market by Jean-Georges,"[French, Asian]",Paris,€31 to €50,4.6,"[[Organic Egg Toast, Caviar and Dill , Wine Pa...","[48.870321, 2.312219]",0.721154,0.626168,0.0


Calcul score

In [25]:
# Set the locale to French
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Get the current date
current_date = datetime.datetime.now()

# Get the name of the current month in French
current_month_name = current_date.strftime("%B")

In [26]:
current_legumes = noms_par_mois[current_month_name]["Legumes"]
current_fruits = noms_par_mois[current_month_name]["Fruits"]
current_cereales = noms_par_mois[current_month_name]["Céréales et légumineuses"]

In [27]:
current_legumes

['Ail',
 'Betterave',
 'Carotte',
 'Céleri-branche',
 'Céleri-rave',
 'Chou',
 'Chou blanc',
 'Chou de Bruxelles',
 'Chou frisé',
 'Chou rouge',
 'Courge',
 'Crosne',
 'Endive',
 'Épinard',
 'Frisée',
 'Mâche',
 'Navet',
 'Oignon',
 'Panais',
 'Poireau',
 'Pomme de terre de conservation',
 'Salsifi',
 'Topinambour']

In [28]:
current_fruits

['Amande sèche',
 'Citron',
 'Clémentine',
 'Kaki',
 'Kiwi',
 'Mandarine',
 'Orange',
 'Pamplemousse',
 'Physalis',
 'Poire',
 'Pomme']

In [29]:
current_cereales

['Lentille']

In [21]:
from googletrans import Translator

In [22]:
translator = Translator()

unique_fruits_combined = []
unique_legumes_combined = []
unique_cereales_combined = []

for word in unique_fruits_list:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    unique_fruits_combined.append(french_name)
    unique_fruits_combined.append(english_name)

for word in unique_legumes_list:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    unique_legumes_combined.append(french_name)
    unique_legumes_combined.append(english_name)

for word in unique_cereales_list:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    unique_cereales_combined.append(french_name)
    unique_cereales_combined.append(english_name)

# Print the combined lists with both French and English names
print("Combined Fruits (French and English):", unique_fruits_combined)
print("Combined Legumes (French and English):", unique_legumes_combined)
print("Combined Cereales (French and English):", unique_cereales_combined)

Combined Fruits (French and English): ['Mûre', 'Blackberry', 'Framboise', 'Raspberry', 'Brugnon', 'Nectarine', 'Noisette', 'Hazelnut', 'Nectarine', 'Nectarine', 'Physalis', 'Physalis', 'Myrtille', 'Blueberry', 'Cerise', 'Cherry', 'Citron', 'Lemon', 'Cassis', 'Cassis', 'Pastèque', 'Watermelon', 'Poire', 'Pear', 'Noix', 'Nut', 'Kaki', 'Khaki', 'Melon', 'Melon', 'Mandarine', 'Mandarin', 'Pêche', 'Fishing', 'Pomme', 'Apple', 'Groseille', 'Currant', 'Abricot', 'Apricot', 'Prune', 'Plum', 'Clémentine', 'Clementine', 'Amande fraiche', 'Fresh almond', 'Châtaigne', 'Chestnut', 'Kiwi', 'Kiwi', 'Baie de goji', 'Goji Berry', 'Figue', 'Fig', 'Pamplemousse', 'Grapefruit', 'Amande sèche', 'Dried almond', 'Rhubarbe', 'Rhubarb', 'Coing', 'Quince', 'Tomate', 'Tomato', 'Raisin', 'Grape', 'Orange', 'Orange', 'Mirabelle', 'Mirabelle', 'Pruneau', 'Prune', 'Fraise', 'Strawberry']
Combined Legumes (French and English): ['Fenouil', 'Fennel', 'Petit pois', 'Pea', 'Salsifi', 'Salsify', 'Ail', 'Garlic', 'Endive',

In [ ]:
translator = Translator()


for word in current_legumes:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    current_legumes.append(english_name)

for word in current_fruits:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    current_fruits.append(english_name)

for word in current_cereales:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    current_cereales.append(english_name)

# Print the combined lists with both French and English names
print("Combined Fruits (French and English):", current_fruits)
print("Combined Legumes (French and English):", current_legumes)
print("Combined Cereales (French and English):", current_cereales)


In [ ]:
translator = Translator()

current_legumes = noms_par_mois[current_month_name]["Legumes"]
current_fruits = noms_par_mois[current_month_name]["Fruits"]
current_cereales = noms_par_mois[current_month_name]["Céréales et légumineuses"]

current_fruits_combined = []
current_legumes_combined = []
current_cereales_combined = []

for word in current_fruits:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    print(english_name)
    current_fruits_combined.append(french_name)
    current_fruits_combined.append(english_name)

for word in current_legumes:
    french_name = word
    print(word)
    english_name = translator.translate(word, src='fr', dest='en').text
    current_legumes_combined.append(french_name)
    current_legumes_combined.append(english_name)

for word in current_cereales:
    french_name = word
    english_name = translator.translate(word, src='fr', dest='en').text
    current_cereales_combined.append(french_name)
    current_cereales_combined.append(english_name)

# Print the combined lists with both French and English names
print("Combined Fruits (French and English):", current_fruits_combined)
print("Combined Legumes (French and English):", current_legumes_combined)
print("Combined Cereales (French and English):", current_cereales_combined)

In [ ]:
current_legumes = noms_par_mois[current_month_name]["Legumes"]
current_fruits = noms_par_mois[current_month_name]["Fruits"]
current_cereales = noms_par_mois[current_month_name]["Céréales et légumineuses"]

print(current_fruits)

In [ ]:
total_fruits_count = 0
total_legumes_count = 0
total_cereales_count = 0

# Iterate through the translated lists and count occurrences
for word in unique_fruits_list:
    total_fruits_count += result_string.lower().count(word.lower())

for word in unique_legumes_list:
    total_legumes_count += result_string.lower().count(word.lower())

for word in unique_cereales_list:
    total_cereales_count += result_string.lower().count(word.lower())

# Print the total counts
print("Total Fruits Count:", total_fruits_count)
print("Total Legumes Count:", total_legumes_count)
print("Total Cereales Count:", total_cereales_count)

In [ ]:
total_fruits_count = 0
total_legumes_count = 0
total_cereales_count = 0

# Iterate through the translated lists and count occurrences
for word in unique_fruits_combined:
    total_fruits_count += result_string.lower().count(word.lower())

for word in unique_legumes_combined:
    total_legumes_count += result_string.lower().count(word.lower())

for word in unique_cereales_combined:
    total_cereales_count += result_string.lower().count(word.lower())

# Print the total counts
print("Total Fruits Count:", total_fruits_count)
print("Total Legumes Count:", total_legumes_count)
print("Total Cereales Count:", total_cereales_count)

In [ ]:
total_current_fruits_count = 0
total_current_legumes_count = 0
total_current_cereales_count = 0

# Iterate through the translated lists and count occurrences
for word in current_fruits:
    total_current_fruits_count += result_string.lower().count(word.lower())

for word in current_legumes:
    total_current_legumes_count += result_string.lower().count(word.lower())

for word in current_cereales:
    total_current_cereales_count += result_string.lower().count(word.lower())

# Print the total counts
print("Total Current Fruits Count:", total_current_fruits_count)
print("Total Current Legumes Count:", total_current_legumes_count)
print("Total Current Cereales Count:", total_current_cereales_count)

In [ ]:
fruit_score = total_current_fruits_count/total_fruits_count
fruit_score

In [ ]:
legume_score = total_current_legumes_count/total_legumes_count
legume_score

In [ ]:
cereales_score = total_current_cereales_count/total_cereales_count
cereales_score